### **코드 설명**

- **문항 매개변수 저장**
  - `item_parameters` 데이터프레임을 생성하여 문항 번호, 변별도(α), 난이도(β)를 저장합니다.
  - `to_csv` 함수를 사용하여 `문항_매개변수.csv` 파일로 저장합니다.
  - `encoding='utf-8-sig'` 옵션을 사용하여 한글이 깨지지 않도록 합니다.

- **수험생 능력 추정치 저장**
  - 개인정보 보호를 위해 필요한 정보만 포함합니다. 여기서는 `이름`과 `능력 추정치`만 저장합니다.
  - 만약 수험생을 식별할 수 있는 민감한 정보(예: 주민등록번호 등)가 있다면 제외하거나 익명화해야 합니다.
  - `examinee_parameters` 데이터프레임을 생성하여 저장합니다.
  - `to_csv` 함수를 사용하여 `수험생_능력_추정치.csv` 파일로 저장합니다.

### **추가 사항**

- **개인정보 보호**
  - 데이터를 저장하거나 공유할 때는 개인정보 보호법 및 관련 규정을 준수해야 합니다.
  - 민감한 개인 식별 정보를 저장하거나 공유할 때는 반드시 적절한 동의를 받아야 하며, 필요에 따라 익명화 또는 비식별화 처리를 해야 합니다.

- **저장된 결과 활용**
  - 저장된 CSV 파일을 불러와서 분석하거나 보고서를 작성할 때 활용할 수 있습니다.
  - 예를 들어, Excel에서 파일을 열어 결과를 확인하거나, 다른 분석 프로그램에서 데이터를 불러올 수 있습니다.

### **CSV 파일 예시**

**문항_매개변수.csv**

| 문항 번호 | 변별도 (alpha) | 난이도 (beta) |
|-----------|----------------|---------------|
| 1         | 1.2345         | -0.5678       |
| 2         | 0.9876         | 0.1234        |
| ...       | ...            | ...           |

**수험생_능력_추정치.csv**

| 이름     | 능력 추정치 (theta) |
|----------|---------------------|
| 홍길동   | 0.4567              |
| 이영희   | -0.1234             |
| ...      | ...                 |

### **참고사항**

- **파일 경로 설정**
  - `to_csv` 함수에서 파일 경로를 지정하지 않으면 현재 작업 디렉토리에 파일이 저장됩니다.
  - 특정 폴더에 저장하고 싶다면 경로를 지정해 주세요. 예: `to_csv('results/문항_매개변수.csv', ...)`

- **다른 형식으로 저장**
  - 필요에 따라 Excel 파일(`.xlsx`)로 저장하거나, 데이터베이스에 저장할 수도 있습니다.
  - Excel로 저장하려면 `pandas`의 `to_excel` 함수를 사용할 수 있습니다.

    ```python
    item_parameters.to_excel('문항_매개변수.xlsx', index=False)
    examinee_parameters.to_excel('수험생_능력_추정치.xlsx', index=False)
    ```

- **추가 분석**
  - 저장된 결과를 활용하여 문항 분석, 수험생 능력 분포 분석 등 추가적인 분석을 수행할 수 있습니다.

### **요약**

- 추정된 문항 매개변수와 수험생의 능력 추정치를 CSV 파일로 저장하였습니다.
- 개인정보 보호를 위해 민감한 정보는 제외하고, 필요한 정보만 포함하였습니다.
- 저장된 결과는 추후 분석이나 보고서 작성 등에 활용할 수 있습니다.

---

In [4]:
import pandas as pd
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import os

# CSV 파일 읽기
df = pd.read_csv('응답_데이터_구술.csv', encoding='utf-8')

# '득점리스트' 문자열을 정수 리스트로 변환
df['responses'] = df['득점리스트'].apply(lambda x: list(map(int, x.split(','))))

# 응답 데이터를 DataFrame으로 생성
response_data = pd.DataFrame(df['responses'].tolist())

# NumPy 배열로 변환
data = response_data.values

# 피험자 수와 문항 수
n_examinees, n_items = data.shape

# 원본 데이터의 최소값과 최대값 확인
print("Original data min:", data.min())
print("Original data max:", data.max())

# 응답 데이터를 0부터 시작하는 범주 인덱스로 조정
data_adjusted = data.astype(int) - data.min()

# 데이터 범위 확인
print("Data adjusted min:", data_adjusted.min())
print("Data adjusted max:", data_adjusted.max())

# 응답 범주 수 계산
num_categories = int(data_adjusted.max()) + 1  # 0부터 시작하므로 +1

# 각 문항당 임계값 수
num_thresholds = num_categories - 1

# PyMC를 사용하여 GRM 모델 구축
with pm.Model() as grm_model:
    # 각 피험자의 능력 모수(theta)
    theta = pm.Normal('theta', mu=0, sigma=1, shape=n_examinees)
    
    # 각 문항의 변별도 모수(alpha)
    alpha = pm.HalfNormal('alpha', sigma=1, shape=n_items)
    
    # 각 문항의 임계값 모수(delta), 정렬되지 않은 상태
    delta_unordered = pm.Normal('delta_unordered', mu=0, sigma=1, shape=(n_items, num_thresholds))
    
    # 각 문항의 임계값을 오름차순으로 정렬
    delta_ordered = pm.Deterministic('delta_ordered', pt.sort(delta_unordered, axis=1))
    
    # 각 범주에 대한 누적 확률 계산
    eta = alpha[None, :, None] * (theta[:, None, None] - delta_ordered[None, :, :])
    p_cumulative = pm.math.sigmoid(eta)
    
    # 각 범주의 확률 계산
    prob = pm.math.concatenate(
        [
            pm.math.ones((n_examinees, n_items, 1)),
            p_cumulative,
            pm.math.zeros((n_examinees, n_items, 1)),
        ],
        axis=2,
    )
    prob = prob[:, :, :-1] - prob[:, :, 1:]
    
    # 확률을 정규화하여 합이 1이 되도록 함
    prob = prob / prob.sum(axis=2, keepdims=True)
    
    # 확률 값이 유효한 범위 내에 있도록 클리핑
    prob = pm.math.clip(prob, 1e-10, 1.0 - 1e-10)
    
    # 관측된 데이터 정의
    observed = pm.Categorical('observed', p=prob, observed=data_adjusted)
    
    # 모델 샘플링
    trace = pm.sample(
        1000,
        tune=1000,
        cores=2,
        random_seed=42,
        return_inferencedata=True,
    )

# 추정된 문항 모수 추출
alpha_est = trace.posterior['alpha'].mean(dim=['chain', 'draw']).values
delta_est = trace.posterior['delta_ordered'].mean(dim=['chain', 'draw']).values

# 문항 난이도 계산 (임계값들의 평균)
item_difficulty = delta_est.mean(axis=1)

# 추정된 피험자 능력 모수 추출
theta_est = trace.posterior['theta'].mean(dim=['chain', 'draw']).values

# 문항 모수 저장
item_parameters = pd.DataFrame({
    '문항 번호': np.arange(1, n_items + 1),
    '변별도 (alpha)': alpha_est,
    '난이도': item_difficulty
})

# 임계값을 DataFrame에 추가
for k in range(num_thresholds):
    item_parameters[f'임계값_{k + 1}'] = delta_est[:, k]

# CSV 파일로 저장
item_parameters.to_csv('문항_매개변수_구술.csv', index=False, encoding='utf-8-sig')

# 피험자 능력 추정치 저장
examinee_parameters = df[['이름', '외국인 등록번호']].copy()
examinee_parameters['능력 추정치 (theta)'] = theta_est

# CSV 파일로 저장
examinee_parameters.to_csv('수험생_능력_추정치_구술.csv', index=False, encoding='utf-8-sig')

# 샘플링 결과 저장
trace.to_netcdf('irt_trace_구술.nc')

# 결과 출력
print('Estimated Item Discrimination Parameters (alpha):')
print(alpha_est)

print('\nEstimated Item Difficulty:')
print(item_difficulty)

print('\nEstimated Item Threshold Parameters (delta):')
print(delta_est)

print('\nEstimated Person Ability Parameters (theta):')
print(theta_est)


Original data min: 0
Original data max: 5
Data adjusted min: 0
Data adjusted max: 5


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [theta, alpha, delta_unordered]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 14 seconds.
There were 134 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Estimated Item Discrimination Parameters (alpha):
[1.10662241 1.03991959 0.90248713 1.50464158 1.57710065]

Estimated Item Difficulty:
[ 0.18457249 -0.16157701 -0.21751602 -0.01789174 -0.04638034]

Estimated Item Threshold Parameters (delta):
[[-1.22959284 -0.37590998 -0.02752064  0.89239833  1.66348758]
 [-1.93437109 -1.01709614 -0.14739543  0.56644851  1.72452911]
 [-1.84411857 -0.90513699 -0.00825605  0.61535251  1.05457902]
 [-1.71895085 -0.44742753  0.35992405  0.67493909  1.04205656]
 [-2.13032404 -0.76731809 -0.03465823  0.76318294  1.93721569]]

Estimated Person Ability Parameters (theta):
[ 0.07834941 -0.65297359  0.63308659  1.02547438 -1.21278029  0.49448409
 -0.07542367 -0.72922874 -0.55692155  0.57996525  0.45653605 -0.19932182
  0.46457709 -0.52707634 -0.27568798  0.88097218  0.41795749 -0.05184849
 -0.69955802 -0.95488248  0.2824283   0.48767391  0.81530902  0.54620747
 -0.39092528  0.80314099  0.63709897 -0.91936672  0.08685245  0.43486636]


In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import pytensor.tensor as pt
from pymc.distributions.transforms import Ordered

# Read the CSV file
df = pd.read_csv('응답_데이터_구술.csv', encoding='utf-8')

# Convert '득점리스트' strings to lists of integers
df['responses'] = df['득점리스트'].apply(lambda x: list(map(int, x.split(','))))

# Create a DataFrame of responses
response_data = pd.DataFrame(df['responses'].tolist())

# Convert response_data to a NumPy array
data = response_data.values

# Number of examinees and items
n_examinees, n_items = data.shape

# Original data min and max
print("Original data min:", data.min())
print("Original data max:", data.max())

# Adjust observed data to be category indices starting from 0
data_adjusted = data.astype(int) - data.min()  # Adjust categories to start from 0

# Verify data ranges
print("Data adjusted min:", data_adjusted.min())
print("Data adjusted max:", data_adjusted.max())

# Number of categories
num_categories = int(data_adjusted.max()) + 1  # Categories start from 0

# Number of thresholds per item
num_thresholds = num_categories - 1

# Build the GRM model using PyMC
with pm.Model() as grm_model:
    # Ability parameters for each examinee
    theta = pm.Normal('theta', mu=0, sigma=1, shape=n_examinees)
    
    # Discrimination parameters for each item
    alpha = pm.HalfNormal('alpha', sigma=1, shape=n_items)
    
    # Threshold parameters for each item (unconstrained)
    delta_unordered = pm.Normal('delta_unordered', mu=0, sigma=1, shape=(n_items, num_thresholds))
    
    # Ensure that thresholds are ordered for each item
    delta_ordered = pm.Deterministic('delta_ordered', pt.sort(delta_unordered, axis=1))
    
    # Compute the cumulative probabilities for each category
    # For each item, compute P(Y ≥ k)
    eta = alpha[None, :, None] * (theta[:, None, None] - delta_ordered[None, :, :])
    p_cumulative = pm.math.sigmoid(eta)
    
    # Compute the probability of each category
    prob = pm.math.concatenate(
        [
            pm.math.ones((n_examinees, n_items, 1)),
            p_cumulative,
            pm.math.zeros((n_examinees, n_items, 1)),
        ],
        axis=2,
    )
    prob = prob[:, :, :-1] - prob[:, :, 1:]
    
    # Normalize probabilities to ensure they sum to 1
    prob = prob / prob.sum(axis=2, keepdims=True)
    
    # Ensure probabilities are within valid range
    prob = pm.math.clip(prob, 1e-10, 1.0 - 1e-10)
    
    # Observed data
    observed = pm.Categorical('observed', p=prob, observed=data_adjusted)
    
    # Sample from the posterior distribution
    trace = pm.sample(
        1000,
        tune=1000,
        cores=2,
        random_seed=42,
        return_inferencedata=True,
    )

# Extract estimated item parameters
alpha_est = trace.posterior['alpha'].mean(dim=['chain', 'draw']).values
delta_est = trace.posterior['delta_ordered'].mean(dim=['chain', 'draw']).values

# Extract estimated person abilities
theta_est = trace.posterior['theta'].mean(dim=['chain', 'draw']).values

# Save item parameters
item_parameters = pd.DataFrame({
    '문항 번호': np.arange(1, n_items+1),
    '변별도 (alpha)': alpha_est
})

# Add thresholds to the DataFrame
for k in range(num_thresholds):
    item_parameters[f'임계값_{k+1}'] = delta_est[:, k]

# Save to CSV
item_parameters.to_csv('문항_매개변수_구술.csv', index=False, encoding='utf-8-sig')

# Save examinee abilities
examinee_parameters = df[['이름', '외국인 등록번호']].copy()
examinee_parameters['능력 추정치 (theta)'] = theta_est

# Save to CSV
examinee_parameters.to_csv('수험생_능력_추정치_구술.csv', index=False, encoding='utf-8-sig')

# Save the trace
trace.to_netcdf('irt_trace_구술.nc')

# Output the results
print('Estimated Item Discrimination Parameters (alpha):')
print(alpha_est)

print('\nEstimated Item Threshold Parameters (delta):')
print(delta_est)

print('\nEstimated Person Ability Parameters (theta):')
print(theta_est)
